<a href="https://colab.research.google.com/github/FG2511/ARE/blob/master/high_level_postprocessing/confusion_matrix_activity_pre_sw.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!mkdir -p local_modules/postprocessing_sw_HL

In [2]:
%%writefile local_modules/postprocessing_sw_HL/__init__.py

import numpy as np
from sklearn.metrics import confusion_matrix

def confusion_matrix_activity_pre_sw(test_label, pred,time,lim_inf_colazione,lim_sup_colazione,lim_inf_pranzo,lim_sup_pranzo,lim_inf_cena, lim_sup_cena):
 
  label_time = (test_label,time)
  pred_time = (pred,time)

  for j in range(0,3):
    if j == 0: 
      lim_inf = lim_inf_colazione
      lim_sup = lim_sup_colazione
      activity = 'colazione'
    if j == 1: 
      lim_inf = lim_inf_pranzo
      lim_sup = lim_sup_pranzo
      activity = 'pranzo'
    if j == 2: 
      lim_inf = lim_inf_cena
      lim_sup = lim_sup_cena
      activity = 'cena'

    #prendo le label che ci indicano l'attività svolta solo nella finestra di tempo compresa tra lim_inf e lim_sup 
    real= []

    i = 0 
    while i < len(label_time[0]):
      if label_time[1][i]> lim_inf and label_time[1][i]<lim_sup:
        real.append(label_time[0][i])
      i = i+1

    #prendo le predizioni che ci indicano l'attività svolta solo nella finestra di tempo compresa tra lim_inf e lim_sup
    pred_sintesi = []

    i = 0 
    while i < len(pred_time[0]):
      if pred_time[1][i]>lim_inf and pred_time[1][i]<lim_sup:
        pred_sintesi.append(pred_time[0][i])
      i = i+1

    #calcolo la matrice di confusione solo per il pasto indicato dalla variabile activity
    print("\nCompute confusion matrix senza sw " + activity)
    y_true = real
    y_true = np.array(y_true,dtype=np.int)
    n_y_pred = pred_sintesi
    tn, fp, fn, tp = confusion_matrix(y_true, n_y_pred).ravel()
    print("TN", tn)
    print("FP", fp)
    print("FN", fn)
    print("TP", tp)
    other = 100*tn/(tn+fp)
    pasto = 100*tp/(fn+tp)
    print("Other corretti: %.2f %%" % other)
    print("Pasto corretti: %.2f %%" % pasto)
    

def sw_for_real_activity(test_label,time,lim_inf_colazione,lim_sup_colazione,lim_inf_pranzo,lim_sup_pranzo,lim_inf_cena,lim_sup_cena,finestra_colazione,finestra_pranzo,finestra_cena):    

  #calcolo le sliding windows per ogni tipologia di pasto (colazione, pranzo, cena) nell'array in cui sono presenti le attività reali 
  #per ogni sliding windows mi salvo il minuto centrale come identificatore dell'attività che si è realmente svolta in quella finestra di tempo

  sw_col_real = []
  sw_pranzo_real = []
  sw_cena_real = []

  for j in range(0,3):
    if j == 0: 
      lim_inf = lim_inf_colazione
      lim_sup = lim_sup_colazione
      finestra = finestra_colazione
    if j == 1: 
      lim_inf = lim_inf_pranzo
      lim_sup = lim_sup_pranzo
      finestra = finestra_pranzo
    if j == 2: 
      lim_inf = lim_inf_cena
      lim_sup = lim_sup_cena
      finestra = finestra_cena

    #estrazione sliding windows 
    pasto = list(more_itertools.windowed(test_label, n=finestra, step=1))
    time_pasto = list(more_itertools.windowed(time, n=finestra, step=1))
    lista = (pasto,time_pasto)

    sw_real = []

    i = 0 
    while i < len(lista[0]):
      if lista[1][i][0]>lim_inf and lista[1][i][finestra-1]<lim_sup:
        centro = int((finestra-1)/2)
        sw_real.append(lista[0][i][centro])
      i = i+1

    if j == 0:
      sw_col_real = sw_real
    if j == 1:
      sw_pranzo_real = sw_real
    if j == 2:
      sw_cena_real = sw_real

  #print(sw_col_real)
  #print(len(sw_col_real))
  #print(sw_pranzo_real)
  #print(len(sw_pranzo_real))
  #print(sw_cena_real)
  #print(len(sw_cena_real))

  sw_real_activity = (sw_col_real,sw_pranzo_real,sw_cena_real)
  return sw_real_activity

def sw_for_pred_activity(pred,time,lim_inf_colazione,lim_sup_colazione,lim_inf_pranzo,lim_sup_pranzo,lim_inf_cena,lim_sup_cena,finestra_colazione,finestra_pranzo,finestra_cena):
  
  #calcolo le sliding windows per ogni tipologia di pasto (colazione, pranzo, cena) nell'array delle predizioni
  sw_col = []
  sw_pranzo = []
  sw_cena = []
  
  for j in range(0,3):
    if j == 0:  
      lim_inf = lim_inf_colazione
      lim_sup = lim_sup_colazione
      finestra = finestra_colazione
    if j == 1: 
      lim_inf = lim_inf_pranzo
      lim_sup = lim_sup_pranzo
      finestra = finestra_pranzo
    if j == 2: 
      lim_inf = lim_inf_cena
      lim_sup = lim_sup_cena
      finestra = finestra_cena

    #estrazione sliding windows
    pred_sws = list(more_itertools.windowed(pred, n=finestra, step=1))
    time_sws = list(more_itertools.windowed(time, n=finestra, step=1))
    lista = (pred_sws,time_sws)

    sw = []

    i = 0 
    while i < len(lista[0]):
      if lista[1][i][0]>lim_inf and lista[1][i][finestra-1]<lim_sup:
        sw.append(lista[0][i])
      i = i+1

    print(len(sw))
    if j == 0:
      sw_col = sw    
    if j == 1:
      sw_pranzo = sw
    if j == 2:
      sw_cena = sw

  #creazione di tre array in cui si inserisce il minuto centrale di una sw se questa ci sembra una colazione, un pranzo o una cena    
  result_col = [];
  result_pranzo = [];
  result_cena = [];

  for j in range(0,3):
    if j == 0: 
      windowsList = sw_col
      min_pasto = min_pasti_colazione
    if j == 1: 
      windowsList = sw_pranzo
      min_pasto = min_pasti_pranzo
    if j == 2: 
      windowsList = sw_cena
      min_pasto = min_pasti_cena

    result = []
    i = 0 

    while i < len(windowsList): 
      count = 0

      z = 0  
      while z < len(windowsList[i]):
        if windowsList[i][z] == 1 :
          count = count + 1
        z = z + 1

      if count > min_pasto:
        result.append(1)
      else:
        result.append(0)

      i = i + 1

    if j == 0:
      result_col = result
    if j == 1:
      result_pranzo = result
    if j == 2:
      result_cena = result

  #print(result_col)
  #print(len(result_col))
  #print(result_pranzo)
  #print(len(result_pranzo))
  #print(result_cena)
  #print(len(result_cena))
  
  sw_pred_activity = (result_col, result_pranzo, result_cena)
  return sw_pred_activity


Writing local_modules/postprocessing_sw_HL/__init__.py
